# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
!pip install -U wikipedia
#!pip install --upgrade langchain
!pip install python-dotenv
#!pip install --upgrade openai
!pip install tiktoken
!pip install docarray
!pip install --upgrade --quiet langchain langchain-openai
!pip install langchain-experimental
!pip install text_generation
!pip install numexpr

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=16d5dd72d4148e96dcd48afa632dfd47c28a95a79ab3d3b27d463bebc2caa5ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-2wzvqlyi/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 5.9 MB/s eta 0:00:00

[notice] A new release of 

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## Built-in LangChain tools

In [3]:
#from langchain.agents.agent_toolkits import create_python_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

#from langchain.tools.python.tool import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [4]:
#llm = ChatOpenAI(temperature=0)

In [5]:
# LLM Inference Server URL
inference_server_url = "http://hf-mistral-7b.llm-model-collection.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [6]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [7]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


: To find out what 25% of 300 is, I need to calculate 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "0.25*300"
}
```
Observation:Thought: To find out what 25% of 300 is, I need to calculate 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "0.25*300"
}
```
Observation:```text
0.25*300
```
...numexpr.evaluate("0.25*300")...
```output
Observation: Answer: 75.0
Thought: I now know the final answer.
Final Answer: 75.0 I now know the final answer.
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [9]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should look up Tom M. Mitchell on Wikipedia to find out what books he wrote.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation:Thought: I should look up Tom M. Mitchell on Wikipedia to find out what books he wrote.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation:
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Scien

## Python Agent

In [10]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [11]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [12]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
 I need to

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 sort the customers by last name and then first name. I can use the `sorted()` function to do this.
Action: Use the `sorted()` function to sort the customers by last name and then first name.
Action Input: `sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))`
Observation: I need to sort the customers by last name and then first name. I can use the `sorted()` function to do this.
Action: Use the `sorted()` function to sort the customers by last name and then first name.
Action Input: `sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))`
Observation:
Observation: Use the `sorted()` function to sort the customers by last name and then first name. is not a valid tool, try one of [Python_REPL].
Thought: I need to sort the customers by last name and 

'Agent stopped due to iteration limit or time limit.'

#### View detailed outputs of the chains

In [13]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFaceTextGenInference] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute pytho

## Define your own tool

In [14]:
!pip install DateTime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 14.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [15]:
from langchain.agents import tool
from datetime import date

In [16]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [17]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [18]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the time tool to get the current date
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation:Thought: I should use the time tool to get the current date
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation:
Observation: 2024-02-29
Thought: I now know the final answer
Final Answer: 2024-02-29 I now know the final answer
Final Answer: 2024-02-29

> Finished chain.
